In [5]:
%matplotlib inline
import glob
import matplotlib.pyplot as plt
import pandas as pd
import pulp
import csv


pd.set_option('display.width', 1000)

In [43]:
# create dataframe of past salaries and past performance
fantasy_full = pd.read_csv("../Data/final.csv")

# create dataframe of past salaries and past performance
fantasyDay = pd.read_csv("../Data/20200209__players.csv")

# Print the first five rows of data to the screen
fantasy_full.head()

,Name,GID,Team,Opp,Home,NewGameFDP,LastFDP,AvgFDP,AvgPTS,LastPT,...,LastTOV,Date,Last3GameAvgFDP,Last3GameAvgMIN,Last3GameAvgPTS,position,Rank,TeamStdFDP,TeamAvgFDP,TeamMaxFDP
0,Louis Williams,3816,lac,gsw,0,40.6,38.5,38.5,21.0,21,...,2,2019-10-23,38.5,36.733333,21.0,SG,advanced,11.726994,30.028571,47.7
1,Montrezl Harrell,4857,lac,gsw,0,23.7,34.4,34.4,17.0,17,...,3,2019-10-23,34.4,38.366667,17.0,C,advanced,11.726994,30.028571,47.7
2,Patrick Beverley,4234,lac,gsw,0,27.3,24.0,24.0,2.0,2,...,2,2019-10-23,24.0,31.366667,2.0,PG,average,11.726994,30.028571,47.7
3,Andre Drummond,4489,det,atl,1,49.4,81.6,81.6,32.0,32,...,2,2019-10-23,81.6,40.800000,32.0,C,top,23.544654,27.325000,81.6
4,Derrick Rose,4073,det,atl,1,29.7,29.9,29.9,18.0,18,...,4,2019-10-23,29.9,26.583333,18.0,SG,average,23.544654,27.325000,81.6


In [44]:
fantasyDay.columns

Index(['Name', 'Last', 'First', 'Player_Name', 'GID', 'Date', 'Starter', 'FD_Pts', 'FD_Sal', 'Pos', 'Team', 'H/A', 'Oppt', 'Team Score', 'Oppt Score', 'Minutes', 'Stat line'], dtype='object')

In [45]:
fantasyDay = fantasyDay[['Player_Name', 'FD_Pts', 'FD_Sal', 'Pos', 'Date']]

In [46]:
fantasyDay = fantasyDay.rename(columns={'Player_Name':'Name'})

In [47]:
fantasy_df = pd.merge(fantasy_full,fantasyDay, on="Name", how="inner")
fantasy_df

,Name,GID,Team,Opp,Home,NewGameFDP,LastFDP,AvgFDP,AvgPTS,LastPT,...,Last3GameAvgPTS,position,Rank,TeamStdFDP,TeamAvgFDP,TeamMaxFDP,FD_Pts,FD_Sal,Pos,Date_y
0,Louis Williams,3816,lac,gsw,0,40.6,38.5,38.50,21.0,21,...,21.000000,SG,advanced,11.726994,30.028571,47.700000,39.5,"5,400",SG,20200209
1,Montrezl Harrell,4857,lac,gsw,0,23.7,34.4,34.40,17.0,17,...,17.000000,C,advanced,11.726994,30.028571,47.700000,36.3,"5,500",C,20200209
2,Patrick Beverley,4234,lac,gsw,0,27.3,24.0,24.00,2.0,2,...,2.000000,PG,average,11.726994,30.028571,47.700000,0.0,"4,800",PG,20200209
3,Andre Drummond,4489,det,atl,1,49.4,81.6,81.60,32.0,32,...,32.000000,C,top,23.544654,27.325000,81.600000,45.3,"10,300",C,20200209
4,Landry Shamet,5704,lac,gsw,0,17.5,14.6,14.60,8.0,8,...,8.000000,PG,belowAvg,11.726994,30.028571,47.700000,23.3,"4,400",PG,20200209
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Kadeem Allen,5113,nyk,bkn,0,-1.0,14.0,14.00,9.0,9,...,9.000000,SG,belowAvg,12.078970,20.701302,43.940000,0.0,"3,500",SG,20200209
164,Gary Payton,4976,was,det,0,31.5,46.7,46.70,10.0,10,...,10.000000,SG,top,13.310857,24.077764,49.868182,0.0,"3,500",SG,20200209
165,Johnathan Williams,5695,was,nyk,1,8.6,24.6,24.60,9.0,9,...,9.000000,PF,average,13.282321,23.999902,49.868182,0.0,"3,700",PF,20200209
166,Brandon Goodwin,5727,atl,bos,0,22.3,40.6,11.78,5.2,21,...,8.666667,PG,belowAvg,14.271173,21.918175,52.321739,3.0,"4,100",PG,20200209


In [48]:
fantasy_df.columns

Index(['Name', 'GID', 'Team', 'Opp', 'Home', 'NewGameFDP', 'LastFDP', 'AvgFDP', 'AvgPTS', 'LastPT', 'AvgMIN', 'LastMIN', 'AvgFGM', 'LastFGM', 'AvgFGA', 'LastFGA', 'Avg3PTS', 'Last3PTS', 'AvgREB', 'LastREB', 'AvgAST', 'LastAST', 'AvgSTL', 'AvgTOV', 'LastTOV', 'Date_x', 'Last3GameAvgFDP', 'Last3GameAvgMIN', 'Last3GameAvgPTS', 'position', 'Rank', 'TeamStdFDP', 'TeamAvgFDP', 'TeamMaxFDP', 'FD_Pts', 'FD_Sal', 'Pos', 'Date_y'], dtype='object')

In [49]:
fantasy_df.corr()['NewGameFDP']

GID               -0.147646
Home               0.000297
NewGameFDP         1.000000
LastFDP            0.550179
AvgFDP             0.622402
AvgPTS             0.572198
LastPT             0.487042
AvgMIN             0.518341
LastMIN            0.488388
AvgFGM             0.124460
LastFGM           -0.006796
AvgFGA             0.178996
LastFGA            0.098782
Avg3PTS            0.122706
Last3PTS           0.002184
AvgREB             0.480165
LastREB            0.415859
AvgAST             0.367484
LastAST            0.307663
AvgSTL             0.209769
AvgTOV             0.427513
LastTOV            0.398534
Last3GameAvgFDP    0.612340
Last3GameAvgMIN    0.521417
Last3GameAvgPTS    0.560939
TeamStdFDP        -0.001768
TeamAvgFDP         0.136338
TeamMaxFDP         0.035109
FD_Pts             0.517957
Date_y                  NaN
Name: NewGameFDP, dtype: float64

In [50]:
to_drop = ['GID', 'Home', 'AvgFGM', 'LastFGM', 'AvgFGA', 'LastFGA', 'Avg3PTS', 'LastREB',  'LastAST', 'AvgSTL', \
           'TeamStdFDP', 'TeamAvgFDP', 'TeamMaxFDP', 'Date_y', 'Last3PTS', 'Team', 'Opp', 'Rank', 'Pos']
fantasy_df.drop(to_drop, axis=1, inplace = True)
fantasy_df

,Name,NewGameFDP,LastFDP,AvgFDP,AvgPTS,LastPT,AvgMIN,LastMIN,AvgREB,AvgAST,AvgTOV,LastTOV,Date_x,Last3GameAvgFDP,Last3GameAvgMIN,Last3GameAvgPTS,position,FD_Pts,FD_Sal
0,Louis Williams,40.6,38.5,38.50,21.0,21,36.733333,36.733333,5.0,7.0,2.0,2,2019-10-23,38.5,36.733333,21.000000,SG,39.5,"5,400"
1,Montrezl Harrell,23.7,34.4,34.40,17.0,17,38.366667,38.366667,7.0,4.0,3.0,3,2019-10-23,34.4,38.366667,17.000000,C,36.3,"5,500"
2,Patrick Beverley,27.3,24.0,24.00,2.0,2,31.366667,31.366667,10.0,6.0,2.0,2,2019-10-23,24.0,31.366667,2.000000,PG,0.0,"4,800"
3,Andre Drummond,49.4,81.6,81.60,32.0,32,40.800000,40.800000,23.0,2.0,2.0,2,2019-10-23,81.6,40.800000,32.000000,C,45.3,"10,300"
4,Landry Shamet,17.5,14.6,14.60,8.0,8,26.433333,26.433333,3.0,2.0,0.0,0,2019-10-23,14.6,26.433333,8.000000,PG,23.3,"4,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Kadeem Allen,-1.0,14.0,14.00,9.0,9,15.533333,15.533333,0.0,2.0,1.0,1,2019-12-25,14.0,15.533333,9.000000,SG,0.0,"3,500"
164,Gary Payton,31.5,46.7,46.70,10.0,10,34.016667,34.016667,11.0,5.0,2.0,2,2019-12-25,46.7,34.016667,10.000000,SG,0.0,"3,500"
165,Johnathan Williams,8.6,24.6,24.60,9.0,9,32.616667,32.616667,8.0,2.0,0.0,0,2019-12-27,24.6,32.616667,9.000000,PF,0.0,"3,700"
166,Brandon Goodwin,22.3,40.6,11.78,5.2,21,10.790000,21.900000,1.4,1.8,0.8,2,2020-01-02,20.3,16.172222,8.666667,PG,3.0,"4,100"


In [51]:
fantasy_df['Name'] = fantasy_df['Name'].replace(' ', '_', regex=True)
fantasy_df

,Name,NewGameFDP,LastFDP,AvgFDP,AvgPTS,LastPT,AvgMIN,LastMIN,AvgREB,AvgAST,AvgTOV,LastTOV,Date_x,Last3GameAvgFDP,Last3GameAvgMIN,Last3GameAvgPTS,position,FD_Pts,FD_Sal
0,Louis_Williams,40.6,38.5,38.50,21.0,21,36.733333,36.733333,5.0,7.0,2.0,2,2019-10-23,38.5,36.733333,21.000000,SG,39.5,"5,400"
1,Montrezl_Harrell,23.7,34.4,34.40,17.0,17,38.366667,38.366667,7.0,4.0,3.0,3,2019-10-23,34.4,38.366667,17.000000,C,36.3,"5,500"
2,Patrick_Beverley,27.3,24.0,24.00,2.0,2,31.366667,31.366667,10.0,6.0,2.0,2,2019-10-23,24.0,31.366667,2.000000,PG,0.0,"4,800"
3,Andre_Drummond,49.4,81.6,81.60,32.0,32,40.800000,40.800000,23.0,2.0,2.0,2,2019-10-23,81.6,40.800000,32.000000,C,45.3,"10,300"
4,Landry_Shamet,17.5,14.6,14.60,8.0,8,26.433333,26.433333,3.0,2.0,0.0,0,2019-10-23,14.6,26.433333,8.000000,PG,23.3,"4,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Kadeem_Allen,-1.0,14.0,14.00,9.0,9,15.533333,15.533333,0.0,2.0,1.0,1,2019-12-25,14.0,15.533333,9.000000,SG,0.0,"3,500"
164,Gary_Payton,31.5,46.7,46.70,10.0,10,34.016667,34.016667,11.0,5.0,2.0,2,2019-12-25,46.7,34.016667,10.000000,SG,0.0,"3,500"
165,Johnathan_Williams,8.6,24.6,24.60,9.0,9,32.616667,32.616667,8.0,2.0,0.0,0,2019-12-27,24.6,32.616667,9.000000,PF,0.0,"3,700"
166,Brandon_Goodwin,22.3,40.6,11.78,5.2,21,10.790000,21.900000,1.4,1.8,0.8,2,2020-01-02,20.3,16.172222,8.666667,PG,3.0,"4,100"


In [52]:
fantasy_df = fantasy_df.dropna()
fantasy_df

,Name,NewGameFDP,LastFDP,AvgFDP,AvgPTS,LastPT,AvgMIN,LastMIN,AvgREB,AvgAST,AvgTOV,LastTOV,Date_x,Last3GameAvgFDP,Last3GameAvgMIN,Last3GameAvgPTS,position,FD_Pts,FD_Sal
0,Louis_Williams,40.6,38.5,38.50,21.0,21,36.733333,36.733333,5.0,7.0,2.0,2,2019-10-23,38.5,36.733333,21.000000,SG,39.5,"5,400"
1,Montrezl_Harrell,23.7,34.4,34.40,17.0,17,38.366667,38.366667,7.0,4.0,3.0,3,2019-10-23,34.4,38.366667,17.000000,C,36.3,"5,500"
2,Patrick_Beverley,27.3,24.0,24.00,2.0,2,31.366667,31.366667,10.0,6.0,2.0,2,2019-10-23,24.0,31.366667,2.000000,PG,0.0,"4,800"
3,Andre_Drummond,49.4,81.6,81.60,32.0,32,40.800000,40.800000,23.0,2.0,2.0,2,2019-10-23,81.6,40.800000,32.000000,C,45.3,"10,300"
4,Landry_Shamet,17.5,14.6,14.60,8.0,8,26.433333,26.433333,3.0,2.0,0.0,0,2019-10-23,14.6,26.433333,8.000000,PG,23.3,"4,400"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
163,Kadeem_Allen,-1.0,14.0,14.00,9.0,9,15.533333,15.533333,0.0,2.0,1.0,1,2019-12-25,14.0,15.533333,9.000000,SG,0.0,"3,500"
164,Gary_Payton,31.5,46.7,46.70,10.0,10,34.016667,34.016667,11.0,5.0,2.0,2,2019-12-25,46.7,34.016667,10.000000,SG,0.0,"3,500"
165,Johnathan_Williams,8.6,24.6,24.60,9.0,9,32.616667,32.616667,8.0,2.0,0.0,0,2019-12-27,24.6,32.616667,9.000000,PF,0.0,"3,700"
166,Brandon_Goodwin,22.3,40.6,11.78,5.2,21,10.790000,21.900000,1.4,1.8,0.8,2,2020-01-02,20.3,16.172222,8.666667,PG,3.0,"4,100"


In [24]:
# fantasy_df = pd.merge(fantasy_df, temp, how='left',on='Name')
# fantasy_df.columns

# LINEAR MODELING

In [36]:
# formulate linear programming problem
problem = pulp.LpProblem("Optimal Lineup", pulp.LpMaximize)

#Defining Variables

In [37]:
# initialize variables
playerPositions = fantasy_df['position'].tolist()
playerNames = fantasy_df['Name'].tolist()
playerCosts = tuple(fantasy_df['FD_Sal'])
playerPoints = tuple(fantasy_df['NewGameFDP'])
salaryCap = 60000
numOfPlayers = range(len(playerNames))

In [38]:
# create variable to represent each player
playerInLineup = pulp.LpVariable.dicts("Player", [i for i in numOfPlayers], 0, 1, cat="Binary")


#Creating the Objective(s)

In [39]:
# Objective: maximize sum of player points in FanDuel
problem += pulp.lpSum(playerInLineup[i] * playerPoints[i] for i in numOfPlayers)
print(problem)

Optimal_Lineup:
MAXIMIZE
40.6*Player_0 + 23.7*Player_1 + 26.9*Player_10 + 19.9*Player_100 + 32.2*Player_101 + 1.2*Player_103 + 26.5*Player_104 + 17.0*Player_105 + 18.3*Player_106 + 7.7*Player_107 + 18.2*Player_108 + 2.0*Player_109 + 24.7*Player_11 + 10.1*Player_110 + 17.9*Player_111 + 8.9*Player_112 + 24.6*Player_113 + 64.1*Player_114 + 22.7*Player_115 + 1.2*Player_116 + 21.3*Player_117 + 18.8*Player_118 + 25.3*Player_119 + 26.3*Player_12 + 13.2*Player_120 + 1.0*Player_121 + 55.3*Player_122 + 15.2*Player_123 + 6.2*Player_124 + 8.7*Player_125 + 7.7*Player_126 + 27.0*Player_128 + 13.0*Player_129 + 16.4*Player_13 + 26.2*Player_130 + 15.2*Player_131 + 24.7*Player_132 + 6.2*Player_133 + 20.1*Player_134 + 28.2*Player_135 + 16.6*Player_136 + 28.8*Player_137 + 16.2*Player_138 + 16.0*Player_139 + 9.6*Player_14 + 17.2*Player_140 + 7.6*Player_141 + 5.6*Player_142 + 17.6*Player_143 + 3.4*Player_144 + 29.3*Player_145 + 52.3*Player_146 + 12.2*Player_147 + 14.2*Player_148 + 34.7*Player_149 + 24.6*Pla

# Defining the Contraints

In [40]:
# constraint n1: each player can only be chosen at most once
for i in numOfPlayers:
    problem += pulp.lpSum(playerInLineup[i]) <= 1
    

In [41]:
# constraint n2: sum of player costs cannot be more than or equal to FanDuel Salary cap ($ 60,000)
problem += sum(playerInLineup[i] * playerCosts[i] for i in numOfPlayers) <= salaryCap

TypeError: unsupported operand type(s) for -=: 'str' and 'int'

In [31]:
# constraint n3: teams can have up to 9 players
problem += pulp.lpSum(problem.variables()) == 9

#Player Positions

In [32]:
# constraint n4: number of PG = 2
pointguards = tuple(fantasy_df['position'] == 'PG')
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * pointguards[i] for i in numOfPlayers) <= 2

In [33]:
# constraint n5: number of SG = 2
shootinguard = tuple(fantasy_df['position'] == 'SG')
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * shootinguard[i] for i in numOfPlayers) <= 2

In [34]:
# constraint n6: number of SF = 2
smallforward = tuple(fantasy_df['position'] == 'SF')
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * smallforward[i] for i in numOfPlayers) <= 2

In [35]:
# constraint n7: number of PF = 2
powerforward = tuple(fantasy_df['position'] == 'PF')
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) >=2
problem += pulp.lpSum(playerInLineup[i] * powerforward[i] for i in numOfPlayers) <= 2

In [36]:
# constraint n8: number of C = 1
center = tuple(fantasy_df['position'] == 'C')
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) >=1
problem += pulp.lpSum(playerInLineup[i] * center[i] for i in numOfPlayers) <= 1

# Solving the Problem

In [37]:
# calculating predicted lineup
cost = 0
points = 0

predicted = set()
if problem.solve() == 1:
    for pos in range(len(numOfPlayers)):
        if playerInLineup[pos].value() == 1:
            name = playerNames[pos]
            predicted.add(name)
            cost += playerCosts[pos]
            points += playerPoints[pos]
            print (playerNames[pos], playerPositions[pos], playerCosts[pos],playerPoints[pos])
    print ('-------------------')
    print (f"The total cost is =: {cost}")
    print (f"The predicted FanDuel points: {points}")
else:
    print ('Error finding solution')

Karl-Anthony_Towns C 10000.0 83.0
RJ_Barrett SG 4900.0 37.1
Delon_Wright SG 4600.0 47.9
LaMarcus_Aldridge PF 6800.0 45.3
Malcolm_Brogdon PG 6700.0 52.4
Giannis_Antetokounmpo SF 11500.0 69.9
Stephen_Curry PG 3500.0 38.1
Omari_Spellman PF 3500.0 35.6
Paul_George SF 8400.0 52.3
-------------------
The total cost is =: 59900.0
The predicted FanDuel points: 461.6000000000001
